In [1]:
import pydot
from collections import defaultdict
from keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D, GRU, Bidirectional
from keras.layers import GlobalMaxPooling1D, LSTM, Dropout, SimpleRNN, TimeDistributed
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.engine.topology import Layer
from keras.layers import concatenate
from keras import activations, initializers, constraints
from keras import regularizers
from keras.regularizers import l1,l2, l1_l2
from keras.callbacks import EarlyStopping
import keras.backend as K
import numpy as np
import os
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
tf.keras.backend.set_session(sess)

from layers import *
from utils import *

Using TensorFlow backend.
C:\Users\weili\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\weili\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\weili\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\weili\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [23]:

## Replace self-attention layers with LSTM layers
def HENIN_LSTM(GCNXss_shape, GCNXpp_shape, reg=l2(5e-4), n_layers=2, n_head=8, size_per_head=8, MAX_REV_LEN=75, MAX_REV_WORD_LEN=10, support=3):
    
    '''
    Comment Encoding
    '''
    
    ''' Capture reviews context correlation'''
    ## word-level encoding
    word_input = Input(shape=(None, 300), dtype='float32')
    #word_sa = Self_Attention(n_head, size_per_head)(word_input)
    word_sa = LSTM(n_head*size_per_head, return_sequences=True)(word_input)
    word_avg = GlobalAveragePooling1D()(word_sa)
    wordEncoder = Model(word_input, word_avg)
    
    ## review-level encoding
    content_input = Input(shape=(MAX_REV_LEN, MAX_REV_WORD_LEN, 300), dtype='float32')
    content_word_encode = TimeDistributed(wordEncoder, name='word_seq_encoder')(content_input)
    #content_sa = Self_Attention(n_head, size_per_head)(content_word_encode)
    content_sa = LSTM(n_head*size_per_head, return_sequences=True)(content_word_encode)
    contentSA_avg_pool = GlobalAveragePooling1D()(content_sa) # session embedding
    
    ''' Capture Post-Comment co-attention'''
    post_words_input = Input(shape=(None, 300), dtype='float32')
    post_lstm = Bidirectional(GRU(32, return_sequences=True))(post_words_input)
    coAtt_vec = CoAttLayer(MAX_REV_LEN)([content_word_encode, post_lstm])
    
    '''
    GCN
    Session-Session Interaction Extractor
    Adjacency: session-session
    '''
    G_ss = [Input(shape=(None, None), batch_shape=(None, None), sparse=True) for _ in range(3)]
    
    X_ss = Input(shape=(GCNXss_shape,))
    X_ss_emb = Dense(16, activation='relu')(X_ss)
    
    # Define GCN model architecture
    H_ss = Dropout(0.2)(X_ss_emb)
    for i in range(n_layers-1):
        H_ss = GraphConvolution(16, support, activation='relu', kernel_regularizer=reg)([H_ss]+G_ss)
        
    H_ss = GraphConvolution(8, support, activation='softmax', kernel_regularizer=reg)([H_ss]+G_ss)
    
    '''
    GCN
    Post-Post Interaction Extractor
    Adjacency: post-post
    '''
    G_pp = [Input(shape=(None, None), batch_shape=(None, None), sparse=True) for _ in range(3)]
    
    X_pp = Input(shape=(GCNXpp_shape,))
    X_pp_emb = Dense(16, activation='relu')(X_pp)
    
    # Define GCN model architecture
    H_pp = Dropout(0.2)(X_pp_emb)
    for i in range(n_layers-1):
        H_pp = GraphConvolution(16, support, activation='relu', kernel_regularizer=reg)([H_pp]+G_pp)
    H_pp = GraphConvolution(8, support, activation='softmax', kernel_regularizer=reg)([H_pp]+G_pp)
     
    '''
    Concatenate Comment Encoding & GCN Embedding
    '''
    H = concatenate([contentSA_avg_pool, coAtt_vec, H_ss, H_pp])
    Y = Dense(1, activation='sigmoid')(H)
    
    # Compile model
    model = Model(inputs=[content_input]+[post_words_input]+[X_ss]+G_ss+[X_pp]+G_pp, outputs=Y)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01))
    model.summary()
    
    return model




In [3]:
'''
Load data
'''
# load preprocessed data
with open('preprocessData/Dat4Model.pickle', 'rb') as f:
    Dat4Model = pickle.load(f)
    
# load multi-hot user vectors of each session
with open('preprocessData/multi_hot_users.pickle', 'rb') as f:
    multi_hot_users = pickle.load(f)  
    
w2v_vec_all = Dat4Model['w2v_vec_all'] # features for HENIN
y_all = Dat4Model['y_all'] # target for HENIN
textFeat_all = Dat4Model['textFeat_all']

MAX_REV_WORD_LEN = w2v_vec_all.shape[2]
MAX_REV_LEN = w2v_vec_all.shape[1]

# word embedding of posted text
postEmb = pad_sequences(w2v_vec_all[:,0,:,:], maxlen=MAX_REV_LEN, dtype='float32', padding='post') 



In [4]:
## cross validating for HENIN model
def HENIN_cv(graph, y, A, model, epochs):
    
    skf = StratifiedKFold(n_splits=5, random_state=9999, shuffle=True)
    iters = 0
    
    for train_index, test_index in skf.split(range(len(y)), y):
        y_train, y_test, train_mask = Mask_y(y=y, train_ix=train_index, test_ix=test_index)
        #y_train, y_test = Mask_y(y=y, train_ix=train_index, test_ix=test_index)
        clf = model
        for epoch in range(epochs):
            # Changed sample weight, need to change back 
            clf.fit(graph, y_train, sample_weight=train_mask, batch_size=A.shape[0], epochs=1)
        preds = (clf.predict(graph, batch_size=A.shape[0])[:,0] >= 0.5).astype(int)
        
        completePerform = metrics(y, preds) # Complete set performance
        generalPerform = metrics(y[test_index], preds[test_index]) # test set performance
        
        try:
            if iters == 1:
                CP = {k: v + [completePerform[k]] for k, v in CP.items()}
                GP = {k: v + [generalPerform[k]] for k, v in GP.items()}
            else:  
                CP = {k: [v] + [completePerform[k]] for k, v in CP.items()}
                GP = {k: [v] + [generalPerform[k]] for k, v in GP.items()}
                iters += 1
        except:
            CP = completePerform
            GP = generalPerform
    
    AvgCP = {k: '{:.3f}'.format(np.mean(v)) for k, v in CP.items()}
    AvgGP = {k: '{:.3f}'.format(np.mean(v)) for k, v in GP.items()}
    
    return AvgCP, AvgGP

OurComResult = {}
OurGenResult = {}



In [18]:
## HENIN
ppA = genAdjacencyMatrix(textFeat_all[:,0,:], 'cosine')
ssA = genAdjacencyMatrix(multi_hot_users, 'cosine')

graph_ss = genGCNgraph(ssA, multi_hot_users)
graph_pp = genGCNgraph(ppA, textFeat_all[:,0,:])

graph = [w2v_vec_all]+[postEmb]+graph_ss+graph_pp



ValueError: Input 0 is incompatible with layer global_average_pooling1d_12: expected ndim=3, found ndim=2

In [25]:
clf = HENIN_LSTM(GCNXss_shape=multi_hot_users.shape[1], 
	        GCNXpp_shape=textFeat_all[:,0,:].shape[1], 
            reg=l2(5e-4), n_layers=4,
	        n_head=8, size_per_head=8, MAX_REV_LEN=MAX_REV_LEN, 
	        MAX_REV_WORD_LEN=MAX_REV_WORD_LEN, support=3)

AvgCP, AvgGP = HENIN_cv(graph=graph, y=y_all, A=ppA, model=clf, epochs=40)
print(AvgGP)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_70 (InputLayer)           (None, 72176)        0                                            
__________________________________________________________________________________________________
input_74 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 16)           1154832     input_70[0][0]                   
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 16)           4816        input_74[0][0]                   
__________________________________________________________________________________________________
dropout_11

Epoch 1/1
2211/2211 [==============================] - 49s 22ms/step - loss: 0.7651
Epoch 1/1
2211/2211 [==============================] - 48s 22ms/step - loss: 0.7208
Epoch 1/1
2211/2211 [==============================] - 41s 19ms/step - loss: 0.6620
Epoch 1/1
2211/2211 [==============================] - 43s 19ms/step - loss: 0.6426
Epoch 1/1
2211/2211 [==============================] - 40s 18ms/step - loss: 0.5401
Epoch 1/1
2211/2211 [==============================] - 42s 19ms/step - loss: 0.6142
Epoch 1/1
2211/2211 [==============================] - 41s 19ms/step - loss: 0.5322
Epoch 1/1
2211/2211 [==============================] - 48s 22ms/step - loss: 0.5087
Epoch 1/1
2211/2211 [==============================] - 61s 28ms/step - loss: 0.5123
Epoch 1/1
2211/2211 [==============================] - 51s 23ms/step - loss: 0.4948
Epoch 1/1
2211/2211 [==============================] - 56s 26ms/step - loss: 0.4443
Epoch 1/1
2211/2211 [==============================] - 44s 20ms/step - loss:

Epoch 1/1
2211/2211 [==============================] - 40s 18ms/step - loss: 0.2704
Epoch 1/1
2211/2211 [==============================] - 39s 18ms/step - loss: 0.2684
Epoch 1/1
2211/2211 [==============================] - 37s 17ms/step - loss: 0.2782
Epoch 1/1
2211/2211 [==============================] - 42s 19ms/step - loss: 0.2501
Epoch 1/1
2211/2211 [==============================] - 36s 17ms/step - loss: 0.2578
Epoch 1/1
2211/2211 [==============================] - 37s 17ms/step - loss: 0.2718
Epoch 1/1
2211/2211 [==============================] - 41s 19ms/step - loss: 0.2411
Epoch 1/1
2211/2211 [==============================] - 43s 19ms/step - loss: 0.2798
Epoch 1/1
2211/2211 [==============================] - 38s 17ms/step - loss: 0.2891
Epoch 1/1
2211/2211 [==============================] - 35s 16ms/step - loss: 0.2463
Epoch 1/1
2211/2211 [==============================] - 31s 14ms/step - loss: 0.3490
Epoch 1/1
2211/2211 [==============================] - 33s 15ms/step - loss:

In [26]:
# serialize model to JSON
lstm_layer = clf.to_json()
with open("lstm_model.json", "w") as json_file:
    json_file.write(lstm_layer)
# serialize weights to HDF5
clf.save_weights("lstm_model.h5")
print("Saved model to disk")

Saved model to disk
